In [6]:
import rawpy
from PIL import Image, ExifTags
from colour_demosaicing import demosaicing_CFA_Bayer_bilinear, demosaicing_CFA_Bayer_Malvar2004, demosaicing_CFA_Bayer_Menon2007
import numpy as np
import matplotlib.pyplot as plt
from photutils.detection import DAOStarFinder
import os
from astropy.io import fits

In [7]:
dir_path = 'M81+82_052424/'
bias_files = [os.path.join(dir_path,f'DSC_{n:04}.NEF') for n in range(116,126)]
dark_files = [os.path.join(dir_path,f'DSC_{n:04}.NEF') for n in range(106,116)]
flat_files = [os.path.join(dir_path,f'DSC_{n:04}.NEF') for n in range(57,70)]
light_files = [os.path.join(dir_path,f'DSC_{n:04}.NEF') for n in range(77,106)]
light_exp = 150 # exposure time in seconds
flat_exp = 12
dark_exp = 150

In [8]:
def display_im(filepath, vmin=None, vmax=None, cmap=None):
    raw = rawpy.imread(filepath)
    if vmin == None:
        vmin = np.min(raw.raw_image)
    if vmax == None:
        vmax = np.max(raw.raw_image)
    if cmap == None:
        cmap = 'gray'
    plt.imshow(raw.raw_image, vmin=vmin, vmax=vmax, cmap=cmap)

In [4]:
bias_frames = np.array([rawpy.imread(file).raw_image for file in bias_files])
dark_frames = np.array([rawpy.imread(file).raw_image for file in dark_files])
flat_frames = np.array([rawpy.imread(file).raw_image for file in flat_files])
light_frames = np.array([rawpy.imread(file).raw_image for file in light_files])


KeyboardInterrupt: 

In [ ]:
plt.imshow(dark_frames[0], vmin=600,vmax=610)

In [ ]:
master_bias = np.median(bias_frames, axis=0)

In [ ]:
dark_frames_debiased = []
for frame in dark_frames:
    frame = frame - master_bias
    dark_frames_debiased.append(frame)
dark_frames_debiased = np.array(dark_frames_debiased)

In [ ]:
plt.imshow(master_bias, vmin=600,vmax=610)

In [ ]:
plt.imshow(dark_frames_debiased[0], vmin=0,vmax=10)

In [ ]:
master_dark = np.median(dark_frames_debiased, axis=0) / dark_exp

In [ ]:
master_dark.shape

In [ ]:
flat_frames_cal = []
for frame in flat_frames:
    frame = frame - master_bias - master_dark*flat_exp
    flat_frames_cal.append(frame)
flat_frames_cal = np.array(flat_frames_cal)

In [ ]:
master_flat = np.median(flat_frames_cal, axis=0)
master_flat = master_flat/np.median(master_flat)

In [ ]:
plt.imshow(master_flat, vmin=0.5,vmax=2)

In [ ]:
hdu = fits.PrimaryHDU(master_bias)
hdu.writeto('master_bias.fits')

In [ ]:
hdu = fits.PrimaryHDU(master_dark)
hdu.writeto('master_dark.fits')

In [ ]:
hdu = fits.PrimaryHDU(master_flat)
hdu.writeto('master_flat.fits')

In [ ]:
processed_lights = []
for i, frame in enumerate(light_frames):
    frame = frame - master_bias - master_dark*light_exp
    frame /= master_flat
    processed_lights.append(frame)
    # hdu = fits.PrimaryHDU(frame)
    # hdu.writeto(os.path.join('M81_82_052424_proc',f'light_proc_{i:03}.fits'))